
# Final Project: Tesla vs GameStop — Stock Price and Revenue Dashboard

**Autor:** _Completa con tu nombre_  
**Curso:** IBM Data Science / Python Project for Data Science

Objetivo: Extraer datos de precio (yfinance) y de ingresos (web‑scraping) para **Tesla (TSLA)** y **GameStop (GME)**, luego construir tableros para comparar **precio** vs **ingresos**.

> Requisitos: `yfinance`, `pandas`, `requests`, `beautifulsoup4`, `lxml`, `plotly`.


In [ ]:

# === 0) Dependencias ===
# Ejecuta esta celda si faltan paquetes
# Nota: en algunos entornos necesitarás reiniciar el kernel tras la instalación.
try:
    import yfinance, pandas, bs4, lxml, plotly
except Exception:
    %pip -q install yfinance pandas requests beautifulsoup4 lxml plotly


In [ ]:

# === 1) Imports y utilidades ===
import re
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.options.display.float_format = '{:,.6f}'.format


In [ ]:

# === 2) Funciones ===

def get_stock_history(ticker: str, period: str = "max") -> pd.DataFrame:
    """
    Descarga historial de precios de cierre para 'ticker' usando yfinance.
    Devuelve DataFrame con columnas ['Date', 'Open','High','Low','Close','Volume',...].
    """
    tk = yf.Ticker(ticker)
    df = tk.history(period=period).reset_index()
    # Normaliza nombre de la columna de fecha a 'Date' si es necesario
    if 'date' in df.columns and 'Date' not in df.columns:
        df = df.rename(columns={'date': 'Date'})
    # Asegura tipos
    df['Date'] = pd.to_datetime(df['Date'])
    return df

def _clean_money_to_float(x: str) -> float:
    """Convierte '$12,345' o '12,345' a float. Maneja valores vacíos o '-' como NaN."""
    if x is None:
        return float('nan')
    s = str(x).strip()
    if s in {"", "-", "—", "N/A"}:
        return float('nan')
    # Quita cualquier símbolo de moneda y separadores
    s = re.sub(r'[^0-9.\-]', '', s)
    try:
        return float(s)
    except ValueError:
        return float('nan')

def scrape_quarterly_revenue(symbol: str, company_slug: str) -> pd.DataFrame:
    """
    Extrae la tabla 'Quarterly Revenue' desde Macrotrends.
    symbol: 'TSLA', 'GME', etc.
    company_slug: 'tesla', 'gamestop', etc. como aparece en Macrotrends URLs.
    Devuelve DataFrame con ['Date','Revenue'].
    """
    url = f"https://www.macrotrends.net/stocks/charts/{symbol}/{company_slug}/revenue"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9"
    }
    html = requests.get(url, headers=headers, timeout=30).text
    soup = BeautifulSoup(html, "lxml")

    # Busca la tabla que contiene "Quarterly Revenue"
    tables = pd.read_html(str(soup))
    target = None
    for t in tables:
        cols = [c.lower() for c in t.columns.astype(str).tolist()]
        if ("date" in cols or "quarter" in cols) and any("revenue" in c for c in cols):
            target = t.copy()
            break
    if target is None:
        raise RuntimeError("No se encontró tabla de ingresos trimestrales en la página de Macrotrends.")

    # Normaliza columnas
    rename_map = {}
    for c in target.columns:
        cl = str(c).strip().lower()
        if cl in {"date", "quarter", "quarter end", "quarter ended"}:
            rename_map[c] = "Date"
        if "revenue" in cl:
            rename_map[c] = "Revenue"
    target = target.rename(columns=rename_map)

    # Filtra y limpia
    keep_cols = [c for c in ["Date", "Revenue"] if c in target.columns]
    df = target[keep_cols].dropna(how="all")
    # Limpieza de valores
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Revenue"] = df["Revenue"].apply(_clean_money_to_float)
    df = df.dropna(subset=["Date", "Revenue"]).sort_values("Date").reset_index(drop=True)

    return df

def make_dashboard(stock_df: pd.DataFrame, revenue_df: pd.DataFrame, title_prefix: str):
    """
    Crea un tablero 2x1 con:
      - Arriba: precio de cierre vs tiempo.
      - Abajo: ingresos trimestrales.
    Muestra y devuelve la figura.
    """
    fig = make_subplots(rows=2, cols=1, shared_xaxes=False,
                        subplot_titles=(f"{title_prefix} — Precio de Cierre", f"{title_prefix} — Ingresos Trimestrales (USD)"))
    # Serie de precio
    fig.add_trace(
        go.Scatter(x=stock_df["Date"], y=stock_df["Close"], mode="lines", name="Close"),
        row=1, col=1
    )
    # Serie de ingresos
    fig.add_trace(
        go.Bar(x=revenue_df["Date"], y=revenue_df["Revenue"], name="Revenue"),
        row=2, col=1
    )
    fig.update_layout(height=800, width=1000, title_text=f"{title_prefix}: Precio vs Ingresos", showlegend=True)
    return fig



## Pregunta 1 — Extraer datos de **Tesla** con yfinance _(2 puntos)_


In [ ]:

# Q1
tesla_ticker = "TSLA"
tesla_data = get_stock_history(tesla_ticker, period="max")
tesla_data.head()



## Pregunta 2 — Extraer **ingresos de Tesla** con web‑scraping _(1 punto)_
Origen recomendado: **Macrotrends**.


In [ ]:

# Q2
tesla_revenue = scrape_quarterly_revenue("TSLA", "tesla")
tesla_revenue.tail()



## Pregunta 3 — Extraer datos de **GameStop** con yfinance _(2 puntos)_


In [ ]:

# Q3
gme_ticker = "GME"
gme_data = get_stock_history(gme_ticker, period="max")
gme_data.head()



## Pregunta 4 — Extraer **ingresos de GameStop** con web‑scraping _(1 punto)_


In [ ]:

# Q4
gme_revenue = scrape_quarterly_revenue("GME", "gamestop")
gme_revenue.tail()



### Nota sobre limpieza de fechas y montos
- `Date` se convierte con `pd.to_datetime(..., errors="coerce")` para evitar errores por formatos mixtos.  
- `Revenue` se limpia con `_clean_money_to_float`, que elimina símbolos y comas.



## Pregunta 5 — Tablero **Tesla**: Precio vs Ingresos _(2 puntos)_


In [ ]:

# Q5
fig_tsla = make_dashboard(tesla_data[["Date","Close"]], tesla_revenue[["Date","Revenue"]], "Tesla (TSLA)")
fig_tsla.show()
# Opcional: guarda el tablero como HTML para adjuntar al envío
fig_tsla.write_html("tesla_dashboard.html")
print("Guardado: tesla_dashboard.html")



## Pregunta 6 — Tablero **GameStop**: Precio vs Ingresos _(2 puntos)_


In [ ]:

# Q6
fig_gme = make_dashboard(gme_data[["Date","Close"]], gme_revenue[["Date","Revenue"]], "GameStop (GME)")
fig_gme.show()
# Opcional: guarda el tablero como HTML
fig_gme.write_html("gamestop_dashboard.html")
print("Guardado: gamestop_dashboard.html")



## Pregunta 7 — Compartir tu Notebook _(2 puntos)_
1. Sube este `.ipynb` a GitHub o compártelo en IBM Skills Network/Colab.  
2. Adjunta capturas de:
   - Head de los DataFrames en Preguntas 1–4.
   - Los dos tableros.
3. Incluye los archivos HTML exportados si corresponde.



## Depuración rápida

- **`ValueError` al convertir fechas**: Usa `pd.to_datetime(col, errors="coerce")` y revisa valores nulos.  
- **Cambios en Macrotrends**: Si cambian los encabezados, ajusta el mapeo de columnas en `scrape_quarterly_revenue`.  
- **Bloqueos de red**: Ejecuta en Colab o un entorno con salida a internet.
